In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# struktura powiązań nazw kolumn tabeli z ich typem danych, zgodnie z zakładką Data
types = {
    'row_id': 'int64',
    'timestamp': 'int64',
    'user_id': 'int32',
    'content_id': 'int16',
    'content_type_id': 'int8',
    'task_container_id': 'int16',
    'user_answer': 'int8',
    'answered_correctly': 'int8',
    'prior_question_elapsed_time': 'float32', 
    'prior_question_had_explanation': 'boolean'
}        

# załadowanie danych, przekazanie struktury types w celu wyeliminowania dynamicznego typowania -> przyspieszenie wczytywania danych
train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', dtype = types) # wszystkie dane ładowane do pamięci
# train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', dtype = types, nrows=1000) # pierwsze 1000 wierszy

print("Dane treningowe:")
print(train.shape)
print(train.info())

In [ ]:
print('Ilość użytkowników: ' + str(train.user_id.nunique()))

In [ ]:
print(str(len(train[train.content_type_id == 0])) + ' wierszy reprezentuje pytania zadane użytkownikom')
print(str(len(train[train.content_type_id == 1])) + ' wierszy reprezentuje udział użytkownika w wykładzie')

In [ ]:
print('W zbiorze istnieje ' + str(train.task_container_id.nunique()) + ' grup pytań/wykładków')
# grupa to np. seria 4 pytań/wykładów, każde pytanie/wykład z danej grupy posiada to samo task_container_id

In [ ]:
print('Możliwe odpowiedzi na zadawane pytania:')
train.user_answer.value_counts()
# dane wskazują, że użytkownik ma do wyboru 4 odpowiedzi (pytania typu ABCD)
# użytkownicy najchętniej wybierali odpowiedzi w kolejności: A, B, D, C
# wartość -1 oznacza brak odpowiedzi (wykład)

In [ ]:
fig = plt.figure(figsize=(13,5))
ts = train.timestamp / 86400000
ts.plot.hist(bins=100)
plt.ylabel("Ilość wystąpięń")
plt.xlabel("Ile dni upłynęło od pierwszej aktywności użyt. w systemie do aktywności reprezentowanej przez dany wiersz")
plt.show()

# ograniczenie do 100 dni
fig = plt.figure(figsize=(13,5))
axes = plt.gca()
axes.set_xlim([0,100])
ts.plot.hist(bins=100)
plt.ylabel("Ilość wystąpięń")
plt.xlabel("Ile dni upłynęło od pierwszej aktywności użyt. w systemie do aktywności reprezentowanej przez dany wiersz")
plt.show()

In [ ]:
# kolumna answered_correctly przyjmuje wartości: 1 - poprwna odpowiedź, 0 - zła odpowiedź, -1 - wykład
# celem zadania jest ustalenie tej wartości dla danych treningowych
no_lectures = train[train.answered_correctly != -1] # wykluczenie wykładów
correct_percent = no_lectures[no_lectures.answered_correctly == 1].shape[0] / no_lectures.shape[0]
wrong_percent = 1 - correct_percent
labels = ' Poprawne odpowiedzi ', ' Złe odpowiedzi '
sizes = [correct_percent, wrong_percent]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%')
plt.title('Jak odpowiadali użytkownicy systemu?')
plt.show()

In [ ]:
no_lectures = train[train.answered_correctly != -1] # wykluczenie wykładów
print('Procent poprawnych odpowiedzi udzielonych na pytania z danej grupy (dla wszystkich użytkowników) wraz z ilością wprowadzonych odpowiedzi')
df = no_lectures.groupby(['task_container_id']).agg({'answered_correctly': ['mean', 'count']})
df = df['answered_correctly'].sort_values(by=['mean', 'count'], ascending=False)
print(df)
print('Najprostsze pytania pochodzą z kategorii: ' + str(df.index[0]))
print('Najtrudniejsze pytania pochodzą z kategorii: ' + str(df.index[-1]))

del no_lectures
del df
        

In [ ]:
no_lectures = train[train.answered_correctly != -1] # wykluczenie wykładów
print('Procent poprawnych odpowiedzi udzielonych na pytania przez danych użytkowników, ilość pytań zadana danemu użytkownikowi')
df = no_lectures.groupby(['user_id']).agg({'answered_correctly': ['mean', 'count']})
df_mean = df['answered_correctly'].sort_values(by=['mean', 'count'], ascending=False)
print(df)
print('Najlepszy uczeń: ' + str(df_mean.index[0]))
print('Najgorszy uczeń: ' + str(df_mean.index[-1]))

df_count = df['answered_correctly'].sort_values(by=['count', 'mean'], ascending=False)
print(' ------ ')
print(df_count)
print('Najwięcej odpowiedzi: ' + str(df_count.index[0]))
print('Najmniej odpowiedzi: ' + str(df_count.index[-1]))
del no_lectures
del df
del df_mean
del df_count

In [ ]:
no_lectures = train[train.answered_correctly != -1] # wykluczenie wykładów
df = no_lectures.groupby(['prior_question_had_explanation']).agg({'answered_correctly': ['mean', 'count']})
print('Wpływ podpowiedzi na odpowiedź')
print(df)

del no_lectures
del df

In [ ]:
# Badanie wpływu momentu zalogowania się na średnią poprawność odpowiedzi
no_lectures = train[train.answered_correctly != -1] # wykluczenie wykładów
days = 100 # ile czasu użytkownik jest w systemie
under_n_days = no_lectures[no_lectures.timestamp <= 86400000 * days] # 86400000 ms = 1 dzień
beyond_n_days = no_lectures[no_lectures.timestamp > 86400000 * days]

correct_under_n_days_percent = under_n_days[under_n_days.answered_correctly == 1].shape[0] / under_n_days.shape[0]
correct_beyond_n_days_percent = beyond_n_days[beyond_n_days.answered_correctly == 1].shape[0] / beyond_n_days.shape[0]

print('Procent poprawnych odpowiedzi dla użytkowników zalogwanych w systemie...')
print('mniej niż ' + str(days) + ' dni: ' + str(correct_under_n_days_percent))
print('więcej niż ' + str(days) + ' dni: ' + str(correct_beyond_n_days_percent))

del no_lectures


In [ ]:
# content_id to klucz obcy dla tabeli questions/lectures, reprezentuje konkretne pytanie lub konkretny wykład
content_ids_amount = train.content_id.nunique()
print('Istnieje ' + str(content_ids_amount) + ' różnych pytań/wykładów')
no_lectures = train[train.content_type_id == 0]
question_amount = no_lectures.content_id.nunique()
print('Istnieje ' + str(question_amount) + ' pytań')
lectures_amount = content_ids_amount - question_amount
print('Istnieje ' + str(lectures_amount) + ' wykładków')

labels = ' pytań ', ' wykładów '
sizes = [question_amount, lectures_amount]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%')
plt.title('Content id dla...')
plt.show()